# Self-attention study

## Simple self-attention

In [1]:
import torch

In [2]:
inputs = torch.tensor(
 [[0.43, 0.15, 0.89], # Your (x^1)
 [0.55, 0.87, 0.66], # journey (x^2)
 [0.57, 0.85, 0.64], # starts (x^3)
 [0.22, 0.58, 0.33], # with (x^4)
 [0.77, 0.25, 0.10], # one (x^5)
 [0.05, 0.80, 0.55]] # step (x^6)
)

In [3]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print('Attention weights: ', attn_weights_2)
print('Attention weights sum: ', attn_weights_2.sum())

Attention weights:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Attention weights sum:  tensor(1.)


In [5]:
contex_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    contex_vec_2 += attn_weights_2[i] * x_i
print(contex_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [6]:
attn_weights = inputs @ inputs.T
attn_weights = torch.softmax(attn_weights, dim=-1)

In [7]:
all_contex_vec = attn_weights @ inputs

In [8]:
all_contex_vec

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Self-attention with weights 

In [9]:
x_2 = inputs[1]
in_dim = inputs.shape[1]
out_dim = 2 

In [10]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(in_dim, out_dim), requires_grad=False)
W_keys = torch.nn.Parameter(torch.rand(in_dim, out_dim), requires_grad=False)
W_values = torch.nn.Parameter(torch.rand(in_dim, out_dim), requires_grad=False)

In [11]:
query_2 = x_2 @ W_query
keys = inputs @ W_keys
values = inputs @ W_values

In [12]:
attn_scores_2 = query_2 @ keys.T
attn_weights_2 = torch.softmax(attn_scores_2/keys.shape[-1]**0.5, dim=-1)

In [13]:
contex_vec_2 = attn_weights_2 @ values

In [14]:
contex_vec_2

tensor([0.3061, 0.8210])

In [15]:
import torch.nn as nn

In [16]:
class SelfAttention_v1(nn.Module): 

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        queries = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)

        contex_vec = attn_weights @ values
        return contex_vec

In [17]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(inputs.shape[1], out_dim)

In [18]:
sa_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

In [19]:
class SelfAttention_v2(nn.Module): 

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key =  nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value =  nn.Linear(d_in, d_out, bias = qkv_bias)

    def forward(self, x):
        queries = self.W_query(x) 
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)

        contex_vec = attn_weights @ values
        return contex_vec

In [20]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(inputs.shape[1], out_dim)

In [21]:
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

In [22]:
sa_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

In [23]:
sa_v1.W_query.data.copy_(sa_v2.W_query.weight.T)
sa_v1.W_key.data.copy_(sa_v2.W_key.weight.T)
sa_v1.W_value.data.copy_(sa_v2.W_value.weight.T)

tensor([[ 0.2526,  0.5191],
        [-0.1415, -0.0852],
        [-0.1962, -0.2043]], grad_fn=<CopyBackwards>)

In [24]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

mask = torch.triu(torch.ones(attn_weights.shape[0], attn_weights.shape[1]), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


## Causal Attention

In [25]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

torch.Size([2, 6, 3])


In [26]:
sa_v2.W_key(batch).shape

torch.Size([2, 6, 2])

In [27]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_len, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_len, context_len), 
            diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape 
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)

        attn_weights = self.dropout(attn_weights)
        contex_vec = attn_weights @ values

        return contex_vec
        

In [28]:
batch.shape

torch.Size([2, 6, 3])

In [29]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(3, 3, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 3])


In [30]:
context_vecs

tensor([[[ 0.3326,  0.5659, -0.3132],
         [ 0.3456,  0.5650, -0.2237],
         [ 0.3440,  0.5604, -0.2000],
         [ 0.3103,  0.4941, -0.1606],
         [ 0.2430,  0.4287, -0.1643],
         [ 0.2648,  0.4316, -0.1375]],

        [[ 0.3326,  0.5659, -0.3132],
         [ 0.3456,  0.5650, -0.2237],
         [ 0.3440,  0.5604, -0.2000],
         [ 0.3103,  0.4941, -0.1606],
         [ 0.2430,  0.4287, -0.1643],
         [ 0.2648,  0.4316, -0.1375]]], grad_fn=<UnsafeViewBackward0>)

## MultiHead Attention

In [31]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_len, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_len, dropout, qkv_bias)
                for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [32]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2

In [33]:
mha = MultiHeadAttentionWrapper(
 d_in, d_out, context_length, 0.0, num_heads=2
)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [34]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_len, dropout, num_heads, qkv_bias=False): 
        super().__init__()
        assert (d_out%num_heads == 0), "d_out must be devisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_len), diagonal=1)
        )

    def forward(self, x): 
        b, num_tokens, d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)

        attn_scores = queries @ keys.transpose(2,3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1,2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        context_vec = self.out_proj(context_vec)
        return context_vec

In [40]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
